## RAG From Screatch: ROUTING

![](/Users/dhavalantala/Desktop/langchain_/Photo/routing.png)

### SetvEnvironment Vars and API Keys

In [2]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_PROJECT'] = 'advanced-rag'
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")

## Part 10: Logical and Semantic routing

Use function-calling classification.

Flow: 

![](/Users/dhavalantala/Desktop/langchain_/Photo/logical_and_semantic_routing.png)

Doc: 

[https://python.langchain.com/docs/use_cases/query_analysis/techniques/routing#routing-to-multiple-indexes](https://python.langchain.com/docs/use_cases/query_analysis/techniques/routing#routing-to-multiple-indexes)

In [3]:
from typing import Literal 

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_groq import ChatGroq


## Data Model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["python_docs", "js_docs", "golang_docs"] = Field(
        ...,
        description="Given a user question choose which datasource would be most relevant for answering their question",
    )

## LLM with function call 
llm = ChatGroq(temperature=0)
structured_llm = llm.with_structured_output(RouteQuery)

# Prompt 
system = """You are an expert at routing a user question to the appropriate data source.

Based on the programming language the question is referring to, route it to the relevant data source."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

# Define router 
router = prompt | structured_llm

Note: we used function calling to produce structured output.

![](/Users/dhavalantala/Desktop/langchain_/Photo/llm_with_structured_output.png)

In [4]:
question = """Why doesn't the following code work:

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(["human", "speak in {language}"])
prompt.invoke("french")
"""

result = router.invoke({"question":question})

In [5]:
result

RouteQuery(datasource='python_docs')

In [6]:
result.datasource

'python_docs'

Once we have this, it is trivial to define a branch that uses `result.datasource`

[https://python.langchain.com/docs/expression_language/how_to/routing](https://python.langchain.com/docs/expression_language/how_to/routing)

In [7]:
def choose_route(result):
    if "python_docs" in result.datasource.lower():
        ### Logic here 
        return "chain for python_docs"
    elif "js_docs" in result.datasource.lower():
        ### Logic here 
        return "chain for js_docs"
    else:
        ### Logic here 
        return "golang_docs"

from langchain_core.runnables import RunnableLambda

full_chain = router | RunnableLambda(choose_route)

In [8]:
full_chain.invoke({"question":question})

'chain for python_docs'

### Semantic Routing

Flow:

![](/Users/dhavalantala/Desktop/langchain_/Photo/semantic_routing.png)

Docs: 

[https://python.langchain.com/docs/expression_language/cookbook/embedding_router](https://python.langchain.com/docs/expression_language/cookbook/embedding_router)

In [10]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

## Two Prompts
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

## Embed Prompts
model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}

hf_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

prompt_templates = [physics_template, math_template]
prompt_embeddings = hf_embeddings.embed_documents(prompt_templates)

# Route question to prompt 
def prompt_router(input):
    # Embed question
    query_embedding = hf_embeddings.embed_query(input["query"])
    # Compute similarity
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    # Chosen prompt 
    print("Using MATH" if most_similar == math_template else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar)


chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | ChatGroq()
    | StrOutputParser()
)

print(chain.invoke("What's a black hole"))

/Users/dhavalantala/Desktop/langchain_/langchainvenv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Using PHYSICS
A black hole is a region in space where the gravitational pull is so strong that nothing, not even light, can escape from it. They are formed when massive stars collapse under their own gravity at the end of their life cycle. The core of the star shrinks, which causes the outer layers of the star to fall inward, creating a black hole. The boundary of the black hole is called the event horizon, which marks the point of no return. Anything that crosses the event horizon will be consumed by the black hole. Black holes are invisible, but their presence can be inferred by observing their effect on nearby matter.


## Rag From Scratch: Query Construction

![](/Users/dhavalantala/Desktop/langchain_/Photo/query_construction.png)

[https://blog.langchain.dev/query-construction/](https://blog.langchain.dev/query-construction/)

[https://blog.langchain.dev/enhancing-rag-based-applications-accuracy-by-constructing-and-leveraging-knowledge-graphs/](https://blog.langchain.dev/enhancing-rag-based-applications-accuracy-by-constructing-and-leveraging-knowledge-graphs/)

## Part 11: Query structuring for metadata filters

In Graphdb we can't provide natural language query, so for that what we do? 
we have to do a query contruction for appropriate to our datasourse. 

Flow: 

![](/Users/dhavalantala/Desktop/langchain_/Photo/query_structuring_for_metadata_filters.png)

Many vectorstores contain metadata fields.

This makes it possible to filter for specific chunks based on metadata. 

Docs:

[https://python.langchain.com/docs/use_cases/query_analysis/techniques/structuring](https://python.langchain.com/docs/use_cases/query_analysis/techniques/structuring)

In [14]:
from langchain_community.document_loaders import YoutubeLoader

docs = YoutubeLoader.from_youtube_url(
    "https://youtu.be/lOdXUVYT69I?si=OlaJFDgATKZE8VhN", add_video_info=True
).load()

docs[0].metadata

{'source': 'lOdXUVYT69I',
 'title': 'GraphRAG - The Most Advanced Futuristic RAG | Introduction, Setup, Working, Testing',
 'description': 'Unknown',
 'view_count': 3848,
 'thumbnail_url': 'https://i.ytimg.com/vi/lOdXUVYT69I/hq720.jpg',
 'publish_date': '2024-07-03 00:00:00',
 'length': 902,
 'author': 'Neural Hacks with Vasanth'}

In [15]:
import datetime
from typing import Literal, Optional, Tuple
from langchain_core.pydantic_v1 import BaseModel, Field

class TutorialSearch(BaseModel):
    """Search over a database of tutorial videos about a software library."""

    content_search: str = Field(
        ...,
        description="Similarity search query applied to video transcripts.",
    )
    title_search: str = Field(
        ...,
        description=(
            "Alternate version of the content search query to apply to video titles. "
            "Should be succinct and only include key words that could be in a video "
            "title."
        ),
    )
    min_view_count: Optional[int] = Field(
        None,
        description="Minimum view count filter, inclusive. Only use if explicitly specified.",
    )
    max_view_count: Optional[int] = Field(
        None,
        description="Maximum view count filter, exclusive. Only use if explicitly specified.",
    )
    earliest_publish_date: Optional[datetime.date] = Field(
        None,
        description="Earliest publish date filter, inclusive. Only use if explicitly specified.",
    )
    latest_publish_date: Optional[datetime.date] = Field(
        None,
        description="Latest publish date filter, exclusive. Only use if explicitly specified.",
    )
    min_length_sec: Optional[int] = Field(
        None,
        description="Minimum video length in seconds, inclusive. Only use if explicitly specified.",
    )
    max_length_sec: Optional[int] = Field(
        None,
        description="Maximum video length in seconds, exclusive. Only use if explicitly specified.",
    )

    def pretty_print(self) -> None:
        for field in self.__fields__:
            if getattr(self, field) is not None and getattr(self, field) != getattr(
                self.__fields__[field], "default", None
            ):
                print(f"{field}: {getattr(self, field)}")

Now, we prompt the LLM to produce queries.

In [16]:
from langchain_core.prompts import ChatPromptTemplate

system = """You are an expert at converting user questions into database queries. \
You have access to a database of tutorial videos about a software library for building LLM-powered applications. \
Given a question, return a database query optimized to retrieve the most relevant results.

If there are acronyms or words you are not familiar with, do not try to rephrase them."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)
llm = ChatGroq()
structured_llm = llm.with_structured_output(TutorialSearch)
query_analyzer = prompt | structured_llm

In [17]:
query_analyzer.invoke({"question": "rag from scratch"}).pretty_print()

content_search: convert user questions into database queries
title_search: tutorial videos about a software library for building LLM-powered applications
earliest_publish_date: 2020-01-01


In [18]:
query_analyzer.invoke(
    {"question": "videos on chat langchain published in 2023"}
).pretty_print()

content_search: video
title_search: langchain
earliest_publish_date: 2023-01-01
latest_publish_date: 2024-01-01


In [19]:
query_analyzer.invoke(
    {"question": "videos that are focused on the topic of chat langchain that are published before 2024"}
).pretty_print()

content_search: chat langchain
title_search: chat langchain
earliest_publish_date: 2022-01-01


In [20]:
query_analyzer.invoke(
    {
        "question": "how to use multi-modal models in an agent, only videos under 5 minutes"
    }
).pretty_print()

content_search: use multi-modal models
title_search: how to use multi-modal
max_length_sec: 300
